In [1]:
%matplotlib inline
project_dir = '../../'
data_dir = project_dir + 'data/'

import sys
sys.path.append(project_dir + 'NN/')

import numpy as np
import importlib
import time
import matplotlib.pyplot as plt
import MLP
import layer
importlib.reload(MLP)
importlib.reload(layer)
from MLP import MLP, MLP_w
from utils.preprocessing import split,StandardScaler
from utils.losses import MSE, MEE

In [2]:
''' FOR THE FIRST TIME
raw_data=np.loadtxt("../../data/TR.csv",delimiter=",")[:,1:]

sort_idx = np.argsort(raw_data[:,11]) ; raw_data = raw_data[sort_idx]
np.random.shuffle(raw_data)
input_data=raw_data[:,:-2]
input_labels=raw_data[:,-2:]

frac_test=0.15
data, test_set, labels, test_labels = split(input_data, input_labels, 
                                            kind="hold_out",frac_training=1-frac_test)
Test_dataset = np.column_stack((test_set,test_labels))
Data_dataset = np.column_stack((data,labels))
np.savetxt('../../data/Test_set.txt',Test_dataset) ; np.savetxt('../../data/Data_set.txt',Data_dataset)
'''
raw_data = np.loadtxt("../../data/Data_set.txt")
data_scaler = StandardScaler() ; labels_scaler = StandardScaler()
raw_data[:,:-2] = data_scaler.fit_transform(raw_data[:,:-2]) 
raw_data[:,-2:] = labels_scaler.fit_transform(raw_data[:,-2:])

#splitting the data from the labels
data=raw_data[:,:-2]
labels=raw_data[:,-2:]

In [3]:
#xavier weight inizialization
def xavier(structure):
    start=np.zeros(len(structure))
    for idx,num in enumerate(structure):
        if idx==0:
            start[idx]=np.sqrt(6)/np.sqrt(structure[idx])
        else:
            start[idx]=np.sqrt(6)/np.sqrt(structure[idx-1]+structure[idx])
    return list(start)

In [4]:
list_lamb = np.logspace(-7,-5,5)  ; print(list_lamb)

[1.00000000e-07 3.16227766e-07 1.00000000e-06 3.16227766e-06
 1.00000000e-05]


In [5]:
import utils.grid_search
importlib.reload(utils.grid_search)
from utils.grid_search import grid_search

# Structure to test: 
n_feat = np.shape(labels)[1]
#model1 = {'structure':[10, n_feat], 'func':['tanh', 'linear'], 'starting_points': xavier([10, n_feat])}
model1 = {'structure':[5,10, n_feat], 'func':["tanh",'relu', 'linear'], 'starting_points': xavier([5,10, n_feat])}
model2 = {'structure':[20,20,n_feat], 'func':['tanh','relu','linear'], 'starting_points': xavier([20,20, n_feat])}
model3 = {'structure':[10,10,n_feat], 'func':['tanh','relu','linear'],'starting_points': xavier([10,10,n_feat])}
#model5 = {'structure':[10,5,n_feat], 'func':['tanh','tanh','linear'],'starting_points': xavier([10,5,n_feat])}
#model6 = {'structure':[10,10,n_feat], 'func':['tanh','tanh','linear'],'starting_points': xavier([10,10,n_feat])}

models = [model1, model2, model3]#, model5, model6]
dict_models = {f'Model{i}': m for i, m in enumerate(models)} 

list_eta = [5e-4]
list_alpha = np.arange(0.1,0.3,0.1)
list_lamb = np.logspace(-6,-5,4) ; list_beta = np.arange(0.7,0.9,0.1)
dict_params = {'eta':list_eta, 'alpha':list_alpha, 'lamb':list_lamb, 'epoch':[10000], 
                'RMSProp' : [True], 'nesterov' : [True] , 'beta' : list_beta , 'verbose' : [False], 'batch_size' : [-1], 'error_threshold' : [0] ,'patience' : [200]}

grid_results = grid_search(MLP_w, dict_models, dict_params, 
                           data, labels, MEE, n_jobs = -1,
                           verbose = 1, kind = 'k_fold', k = 3)

start training 72 possible combinations
[trained=72] [elaps t=575.9 s] [remain t =0.0 s]s]
Time for the grid search: 575.8696660995483 s


In [6]:
grid_results[0]
#np.savetxt('../../data/sigmoid_search.txt',grid_results[1:10])

{'train': {'eta': 0.0005,
  'alpha': 0.2,
  'lamb': 2.1544346900318822e-06,
  'epoch': 10000,
  'RMSProp': True,
  'nesterov': True,
  'beta': 0.7999999999999999,
  'verbose': False,
  'batch_size': -1,
  'error_threshold': 0,
  'patience': 200},
 'model': {'structure': [5, 10, 2],
  'func': ['tanh', 'relu', 'linear'],
  'starting_points': [1.0954451150103321,
   0.6324555320336758,
   0.7071067811865475]},
 'model_name': 'Model0',
 'Error': 0.2740324333500792}

In [7]:
#Comment after the grid search is ended
import json

with open('grid_search_pv2.json','w') as fp:
    json.dump(grid_results,fp) 


In [8]:
#loading dictionary grid_search
with open('../report/tanh_search.json') as json_file:
    grid_results = json.load(json_file)

FileNotFoundError: [Errno 2] No such file or directory: '../report/tanh_search.json'

In [ ]:
frac_training=0.8
input_data, val_data, train_labels, val_labels=split(data, labels,kind="hold_out",
                                               frac_training=frac_training)
best_model_dict = grid_results[0]
best_model_dict['model']
best_model = MLP(**best_model_dict['model'])
train_dict = dict(best_model_dict['train'])
#del train_dict['n_candidate']
#del train_dict['test_more_init']

In [ ]:
best_model.train(input_data, train_labels, val_data, val_labels, **train_dict)

In [ ]:
def plot_results(network, input_data, val_data, labels, val_labels):
    train_pred = network.predict(input_data)
    val_pred = network.predict(val_data)
    x = np.arange(len(network.train_MEE))

    fig = plt.figure(figsize=(13,4))

    fig.add_subplot(131)
    plt.plot(x,network.train_MEE)
    plt.plot(x,network.val_MEE,label="test")
    plt.title("Learning curve")
    plt.xlabel("Epochs")
    plt.ylabel("Squared error")
    plt.yscale("log")
    plt.legend()

    fig.add_subplot(132)
    plt.title('Residual for training data')
    plt.plot(np.arange(len(labels)),labels[:,0]-train_pred[:,0],".",label="residual0")
    plt.plot(np.arange(len(labels)),labels[:,1]-train_pred[:,1],".",label="residual1")
    plt.legend()

    fig.add_subplot(133)
    plt.title('Residual for validation data')
    plt.plot(np.arange(len(val_labels)),val_labels[:,0]-val_pred[:,0],".",label="residual0")
    plt.plot(np.arange(len(val_labels)),val_labels[:,1]-val_pred[:,1],".",label="residual1")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_results(best_model, input_data, val_data, train_labels, val_labels)
pred = best_model.predict(data)
plt.plot(labels[:,1],labels[:,0],".")
plt.plot(pred[:,1],pred[:,0],".")


## Testing PARALLEL

In [ ]:
import utils.grid_search
importlib.reload(utils.grid_search)
from utils.grid_search import grid_search

# Structure to test: 
n_feat = np.shape(labels)[1]
model1 = {'structure':[10, n_feat], 'func':['sigmoid', 'linear'], 'starting_points': [0.5]*2}
model2 = {'structure':[15, n_feat], 'func':['sigmoid', 'linear'], 'starting_points': [0.5]*2}
model3 = {'structure':[20, n_feat], 'func':['sigmoid', 'linear'], 'starting_points': [0.5]*2}
#model5= {'structure':[4,4, n_feat], 'func':['sigmoid','sigmoid','linear'], 'starting_points': [0.5]*2}
#model4 = {'structure':[5,4,n_feat], 'func':['sigmoid','sigmoid','linear'],'starting_points': [0.5]*2}
#model6 = {'structure':[6,6,n_feat], 'func':['sigmoid','sigmoid','linear'],'starting_points': [0.5]*2}
models = [model1, model2, ]#model3, model4, model5, model6]
dict_models = {f'Model{i}': m for i, m in enumerate(models)} 

list_eta = [1e-2,5e-3]
list_alpha = [0., 1.]#np.arange(0,1, 0.5)
list_lamb = [0.] # np.arange(0,1,0.5)
list_beta = np.arange(0.6, 1, 0.1)

dict_params = {'eta':list_eta, 'alpha':list_alpha, 'lamb':list_lamb, 'epoch':[1000], 
               'RMSProp' : [True], 'nesterov' : [False] , 'beta' : list_beta, 'verbose': [False] }

grid_results = grid_search(MLP_w, dict_models, dict_params, 
                           data, labels, MEE,
                           n_jobs = -1,
                           verbose = 1, kind = 'hold_out')

In [ ]:
grid_results